In [1]:
import json
import pandas as pd
from utils import choose_best_threshold

In [2]:
excel_file = pd.read_excel("data/AggreFact-Unified-test.xlsx", sheet_name=None)
origin_names = ['cnn', 'xsum']
sheet_names = ['in_pred', 'in_np', 'ex_pred', 'ex_np']
xsum_sheet_names= ['in_sent', 'ex_sent']


from utils import choose_best_threshold
import warnings
warnings.filterwarnings('ignore')

score_df = pd.read_csv("data/aggre_fact_final_main-100.csv")
score_df = score_df.drop(labels=['Unnamed: 0'], axis=1)
#score_df.to_csv("data/aggre_fact_final_main-100.csv")

dataset_list = ['XSumFaith', 'Polytope', 'FactCC', 'SummEval', 'FRANK', 'Wang20', 'CLIFF', 'Cao22', 'Goyal21']
systems = ['DAE', 'QuestEval', 'SummaC-ZS', 'SummaC-Conv', 'QAFactEval', 
           'sbert', 'bert'
           ]
origins = ['cnndm', 'xsum']
origin_mapping = {0: "cnndm", 1: "xsum", 2: "overall"}

for system in systems:
    score_df[f'{system}_label'] = None
print(score_df.columns)

Index(['dataset', 'origin', 'id', 'doc', 'summary', 'model_name', 'label',
       'cut', 'DAE_score', 'DAE_label', 'QuestEval_score', 'QuestEval_label',
       'SummaC-ZS_score', 'SummaC-ZS_label', 'SummaC-Conv_score',
       'SummaC-Conv_label', 'QAFactEval_score', 'QAFactEval_label',
       'sbert_score', 'combine_pos_score', 'bert_score', 'sbert_label',
       'bert_label'],
      dtype='object')


In [ ]:
# Error Type Analysis on All data

score_val = score_df[(score_df.cut == 'val') & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]

for system in systems:
    #for origin in origins:
        #for dataset in dataset_list:
        #    df_val_temp = score_val[(score_val.origin == origin) & (score_val.dataset == dataset)]
        #    df_test_temp = score_test[(score_test.origin == origin) & (score_test.dataset == dataset)]
    df_val_temp = score_val[]
    df_test_temp = score_test[]

    if len(df_val_temp) > 0 and len(df_test_temp) > 0:
        best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)

        scores_test = df_test_temp[f'{system}_score'].values
        preds_test = [1 if score > best_thresh else 0 for score in scores_test]
        
        score_df.loc[df_test_temp.index, f'{system}_label'] = preds_test

In [ ]:
results_dict = {}
from sklearn.metrics import recall_score, precision_score, f1_score

sheet_name = ['in_pred', 'in_np', 'ex_pred', 'ex_np']
for sheet_name in sheet_names:
    origin_sheet_name = f'cnn_{sheet_name}'
    
    excel_file[origin_sheet_name].rename(columns={'summ': 'summary'}, inplace=True)
    excel_file[origin_sheet_name]['id'] = excel_file[origin_sheet_name]['id'].astype(str)

    df = excel_file[origin_sheet_name]
    df.rename(columns={'label': f'{sheet_name}_label'}, inplace=True)
    df['dataset'] = df['dataset'].str.lower()
    score_df['dataset'] = score_df['dataset'].str.lower()
    temp_score_df = score_df[score_df.origin == 'cnndm']
    
    result_df = pd.merge(temp_score_df, df, how='outer', on=['id', 'cut', 'summary', 'dataset', 'model_name'])
    result_df[f'{sheet_name}_label'].fillna(1, inplace=True)
    recalls = []
    for system in systems:
        test_df = result_df[result_df.cut=='test'].dropna()
        recall = recall_score(test_df[f'{sheet_name}_label'].astype(int).tolist(), test_df[f"{system}_label"].tolist(), pos_label=0)
        recalls.append(recall)
    results_dict[sheet_name] = recalls

recall_table = pd.DataFrame(
    results_dict,
    columns=sheet_names,
    index=systems
)
display(recall_table)

In [3]:
# Draw Table 5!! Use the unified data

score_val = score_df[(score_df.cut == 'val') & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]

for system in systems:
    for origin in origins:
        #for dataset in dataset_list:
        #    df_val_temp = score_val[(score_val.origin == origin) & (score_val.dataset == dataset)]
        #    df_test_temp = score_test[(score_test.origin == origin) & (score_test.dataset == dataset)]
        df_val_temp = score_val[(score_val.origin == origin)]
        df_test_temp = score_test[(score_test.origin == origin)]

        if len(df_val_temp) > 0 and len(df_test_temp) > 0:
            best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)

            scores_test = df_test_temp[f'{system}_score'].values
            preds_test = [1 if score > best_thresh else 0 for score in scores_test]
            
            score_df.loc[df_test_temp.index, f'{system}_label'] = preds_test

In [4]:
from sklearn.metrics import recall_score, precision_score, f1_score


sheet_name = ['in_pred', 'in_np', 'ex_pred', 'ex_np']
results_dict = {}
for sheet_name in sheet_names:
    origin_sheet_name = f'cnn_{sheet_name}'
    
    excel_file[origin_sheet_name].rename(columns={'summ': 'summary'}, inplace=True)
    excel_file[origin_sheet_name]['id'] = excel_file[origin_sheet_name]['id'].astype(str)

    df = excel_file[origin_sheet_name]
    df.rename(columns={'label': f'{sheet_name}_label'}, inplace=True)
    df['dataset'] = df['dataset'].str.lower()
    score_df['dataset'] = score_df['dataset'].str.lower()
    temp_score_df = score_df[score_df.origin == 'cnndm']
    
    result_df = pd.merge(temp_score_df, df, how='outer', on=['id', 'cut', 'summary', 'dataset', 'model_name'])
    result_df[f'{sheet_name}_label'].fillna(1, inplace=True)
    recalls = []
    for system in systems:
        test_df = result_df[result_df.cut=='test'].dropna()
        recall = recall_score(test_df[f'{sheet_name}_label'].astype(int).tolist(), test_df[f"{system}_label"].tolist(), pos_label=0)
        recalls.append(recall)
    results_dict[sheet_name] = recalls

recall_table = pd.DataFrame(
    results_dict,
    columns=sheet_names,
    index=systems
)
display(recall_table)    

xsum_sheet_names = ['in_pred', 'in_np', 'in_sent', 'ex_pred', 'ex_np', 'ex_sent']
results_dict = {}
for sheet_name in xsum_sheet_names:
    origin_sheet_name = f'xsum_{sheet_name}'

    excel_file[origin_sheet_name].rename(columns={'summ': 'summary'}, inplace=True)
    excel_file[origin_sheet_name]['id'] = excel_file[origin_sheet_name]['id'].astype(str)

    df = excel_file[origin_sheet_name]
    df.rename(columns={'label': f'{sheet_name}_label'}, inplace=True)
    df['dataset'] = df['dataset'].str.lower()
    score_df['dataset'] = score_df['dataset'].str.lower()
    df["origin"].fillna("xsum", inplace=True)
    df["dataset"].replace("goyaldurrett2021", "goyal21", inplace=True)
    temp_score_df = score_df[score_df.origin == 'xsum']

    result_df = pd.merge(temp_score_df, df, how='outer', on=['id', 'summary', 'dataset', 'cut', 'model_name'])
    result_df[f'{sheet_name}_label'].fillna(1, inplace=True)
    recalls = []
    for system in systems:
        test_df = result_df[result_df.cut=='test'].dropna()
        recall = recall_score(test_df[f'{sheet_name}_label'].tolist(), test_df[f"{system}_label"].tolist(), pos_label=0)
        recalls.append(recall)
    results_dict[sheet_name] = recalls
        

recall_table = pd.DataFrame(
    results_dict,
    columns=xsum_sheet_names,
    index=systems
)
display(recall_table)

,in_pred,in_np,ex_pred,ex_np
DAE,0.563636,0.603448,0.632812,0.686364
QuestEval,0.563636,0.609195,0.609375,0.690909
SummaC-ZS,0.545455,0.545977,0.609375,0.695455
SummaC-Conv,0.781818,0.689655,0.710938,0.845455
QAFactEval,0.509091,0.522989,0.617188,0.772727
sbert,0.800000,0.741379,0.875000,0.900000
bert,0.690909,0.718391,0.812500,0.809091


,in_pred,in_np,in_sent,ex_pred,ex_np,ex_sent
DAE,0.552,0.572139,0.923077,0.793578,0.760956,0.953191
QuestEval,0.648,0.641791,0.923077,0.761468,0.639442,0.919149
SummaC-ZS,0.936,0.935323,1.000000,0.940367,0.946215,0.957447
SummaC-Conv,0.464,0.358209,0.307692,0.444954,0.446215,0.617021
QAFactEval,0.800,0.761194,0.923077,0.766055,0.649402,0.876596
sbert,0.608,0.462687,0.692308,0.573394,0.482072,0.765957
bert,0.800,0.771144,1.000000,0.834862,0.794821,0.889362


# Single Threshold
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>in_pred</th>
      <th>in_np</th>
      <th>ex_pred</th>
      <th>ex_np</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.563636</td>
      <td>0.603448</td>
      <td>0.632812</td>
      <td>0.686364</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.563636</td>
      <td>0.609195</td>
      <td>0.609375</td>
      <td>0.690909</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.545455</td>
      <td>0.545977</td>
      <td>0.609375</td>
      <td>0.695455</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.781818</td>
      <td>0.689655</td>
      <td>0.710938</td>
      <td>0.845455</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.509091</td>
      <td>0.522989</td>
      <td>0.617188</td>
      <td>0.772727</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.800000</td>
      <td>0.741379</td>
      <td>0.875000</td>
      <td>0.900000</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.690909</td>
      <td>0.718391</td>
      <td>0.812500</td>
      <td>0.809091</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>in_pred</th>
      <th>in_np</th>
      <th>in_sent</th>
      <th>ex_pred</th>
      <th>ex_np</th>
      <th>ex_sent</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.552</td>
      <td>0.572139</td>
      <td>0.923077</td>
      <td>0.793578</td>
      <td>0.760956</td>
      <td>0.953191</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.648</td>
      <td>0.641791</td>
      <td>0.923077</td>
      <td>0.761468</td>
      <td>0.639442</td>
      <td>0.919149</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.936</td>
      <td>0.935323</td>
      <td>1.000000</td>
      <td>0.940367</td>
      <td>0.946215</td>
      <td>0.957447</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.464</td>
      <td>0.358209</td>
      <td>0.307692</td>
      <td>0.444954</td>
      <td>0.446215</td>
      <td>0.617021</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.800</td>
      <td>0.761194</td>
      <td>0.923077</td>
      <td>0.766055</td>
      <td>0.649402</td>
      <td>0.876596</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.608</td>
      <td>0.462687</td>
      <td>0.692308</td>
      <td>0.573394</td>
      <td>0.482072</td>
      <td>0.765957</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.800</td>
      <td>0.771144</td>
      <td>1.000000</td>
      <td>0.834862</td>
      <td>0.794821</td>
      <td>0.889362</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cnndm</th>
      <th>xsum</th>
      <th>all</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.784862</td>
      <td>0.757256</td>
      <td>0.873665</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.727684</td>
      <td>0.654354</td>
      <td>0.874377</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.700946</td>
      <td>0.218997</td>
      <td>0.842705</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.693130</td>
      <td>0.720317</td>
      <td>0.750178</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.862608</td>
      <td>0.722955</td>
      <td>0.874733</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.665981</td>
      <td>0.675462</td>
      <td>0.752669</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.768819</td>
      <td>0.686016</td>
      <td>0.761922</td>
    </tr>
  </tbody>
</table>
</div>

# multiple threshold
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>in_pred</th>
      <th>in_np</th>
      <th>ex_pred</th>
      <th>ex_np</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.509091</td>
      <td>0.574713</td>
      <td>0.609375</td>
      <td>0.668182</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.581818</td>
      <td>0.695402</td>
      <td>0.742188</td>
      <td>0.777273</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.745455</td>
      <td>0.632184</td>
      <td>0.710938</td>
      <td>0.800000</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.781818</td>
      <td>0.683908</td>
      <td>0.710938</td>
      <td>0.840909</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.509091</td>
      <td>0.545977</td>
      <td>0.632812</td>
      <td>0.790909</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.436364</td>
      <td>0.454023</td>
      <td>0.562500</td>
      <td>0.586364</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.636364</td>
      <td>0.660920</td>
      <td>0.718750</td>
      <td>0.740909</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>in_pred</th>
      <th>in_np</th>
      <th>in_sent</th>
      <th>ex_pred</th>
      <th>ex_np</th>
      <th>ex_sent</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.584</td>
      <td>0.606965</td>
      <td>0.923077</td>
      <td>0.834862</td>
      <td>0.796813</td>
      <td>0.961702</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.528</td>
      <td>0.487562</td>
      <td>0.923077</td>
      <td>0.637615</td>
      <td>0.500000</td>
      <td>0.855319</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.640</td>
      <td>0.656716</td>
      <td>0.769231</td>
      <td>0.564220</td>
      <td>0.517928</td>
      <td>0.514894</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.648</td>
      <td>0.567164</td>
      <td>0.384615</td>
      <td>0.614679</td>
      <td>0.635458</td>
      <td>0.710638</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.720</td>
      <td>0.686567</td>
      <td>0.846154</td>
      <td>0.646789</td>
      <td>0.539841</td>
      <td>0.804255</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.664</td>
      <td>0.532338</td>
      <td>0.769231</td>
      <td>0.678899</td>
      <td>0.567729</td>
      <td>0.817021</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.632</td>
      <td>0.557214</td>
      <td>0.846154</td>
      <td>0.651376</td>
      <td>0.613546</td>
      <td>0.778723</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cnndm</th>
      <th>xsum</th>
      <th>all</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>DAE</th>
      <td>0.859317</td>
      <td>0.749340</td>
      <td>0.844484</td>
    </tr>
    <tr>
      <th>QuestEval</th>
      <td>0.697244</td>
      <td>0.525066</td>
      <td>0.674021</td>
    </tr>
    <tr>
      <th>SummaC-ZS</th>
      <td>0.774167</td>
      <td>0.617414</td>
      <td>0.753025</td>
    </tr>
    <tr>
      <th>SummaC-Conv</th>
      <td>0.738791</td>
      <td>0.643799</td>
      <td>0.725979</td>
    </tr>
    <tr>
      <th>QAFactEval</th>
      <td>0.878239</td>
      <td>0.649077</td>
      <td>0.847331</td>
    </tr>
    <tr>
      <th>sbert</th>
      <td>0.785274</td>
      <td>0.715040</td>
      <td>0.775801</td>
    </tr>
    <tr>
      <th>bert</th>
      <td>0.732620</td>
      <td>0.480211</td>
      <td>0.698577</td>
    </tr>
  </tbody>
</table>
</div>

In [5]:
score_val = score_df[(score_df.cut == 'val') & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]

for system in systems:
    for origin in origins:
    #    for dataset in dataset_list:
    #        df_val_temp = score_val[(score_val.origin == origin) & (score_val.dataset == dataset)]
    #        df_test_temp = score_test[(score_test.origin == origin) & (score_test.dataset == dataset)]
        df_val_temp = score_val[(score_val.origin == origin)]
        df_test_temp = score_test[(score_test.origin == origin)]

        if len(df_val_temp) > 0 and len(df_test_temp) > 0:
            best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)

            scores_test = df_test_temp[f'{system}_score'].values
            preds_test = [1 if score > best_thresh else 0 for score in scores_test]
            
            score_df.loc[df_test_temp.index, f'{system}_label'] = preds_test

In [6]:
# Recall of correct summaries
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'goyal21'))]
cnn_score_test_pos = score_test[(score_test.origin == 'cnndm')]
xsum_score_test_pos = score_test[(score_test.origin == 'xsum')]

results_dict = {}

recalls = []
for system in systems:
    system_labels = cnn_score_test_pos[f'{system}_label'].values.tolist()
    labels = cnn_score_test_pos['label'].values.tolist()
    recall = recall_score(labels, system_labels)
    recalls.append(recall)

results_dict['cnndm'] = recalls

recalls = []
for system in systems:
    system_labels = xsum_score_test_pos[f'{system}_label'].values.tolist()
    labels = xsum_score_test_pos['label'].values.tolist()
    recall = recall_score(labels, system_labels)
    recalls.append(recall)

results_dict['xsum'] = recalls
    
recall_table = pd.DataFrame(
    results_dict,
    columns=["cnndm", "xsum"],
    index=systems
)
display(recall_table)

,cnndm,xsum
DAE,0.784862,0.757256
QuestEval,0.727684,0.654354
SummaC-ZS,0.700946,0.218997
SummaC-Conv,0.693130,0.720317
QAFactEval,0.862608,0.722955
sbert,0.665981,0.675462
bert,0.768819,0.686016


In [7]:
# Draw Table 5!! Use the unified data

score_val = score_df[(score_df.cut == 'val') & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'Goyal21'))]

for system in systems:
    #for origin in origins:
    #    for dataset in dataset_list:
    #        df_val_temp = score_val[(score_val.origin == origin) & (score_val.dataset == dataset)]
    #        df_test_temp = score_test[(score_test.origin == origin) & (score_test.dataset == dataset)]
    df_val_temp = score_val
    df_test_temp = score_test

    if len(df_val_temp) > 0 and len(df_test_temp) > 0:
        best_thresh, best_f1 = choose_best_threshold(df_val_temp.label.values, df_val_temp[f'{system}_score'].values)

        scores_test = df_test_temp[f'{system}_score'].values
        preds_test = [1 if score > best_thresh else 0 for score in scores_test]
        
        score_df.loc[df_test_temp.index, f'{system}_label'] = preds_test

In [8]:
score_test = score_df[(score_df.cut == 'test')  & ((score_df.origin != 'cnndm') | (score_df.dataset != 'goyal21'))]
score_test_pos = score_test

recalls = []
for system in systems:
    system_labels = score_test_pos[f'{system}_label'].values.tolist()
    labels = score_test_pos['label'].values.tolist()
    recall = recall_score(labels, system_labels)
    recalls.append(recall)

results_dict['all'] = recalls

recall_table = pd.DataFrame(
    results_dict,
    columns=["cnndm", "xsum", 'all'],
    index=systems
)
display(recall_table)

,cnndm,xsum,all
DAE,0.784862,0.757256,0.873665
QuestEval,0.727684,0.654354,0.874377
SummaC-ZS,0.700946,0.218997,0.842705
SummaC-Conv,0.693130,0.720317,0.750178
QAFactEval,0.862608,0.722955,0.874733
sbert,0.665981,0.675462,0.752669
bert,0.768819,0.686016,0.761922
